In [3]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt
from qutip.ipynbtools import version_table
from tqdm.notebook import tqdm
from matplotlib.animation import FuncAnimation
from scipy.linalg import expm

resol = 200
pi = np.pi
version_table()

Software,Version
QuTiP,5.1.1
Numpy,1.26.2
SciPy,1.11.3
matplotlib,3.7.3
Number of CPUs,11
BLAS Info,Generic
IPython,8.17.2
Python,"3.12.0 (v3.12.0:0fb18b02c8, Oct 2 2023, 09:45:56) [Clang 13.0.0 (clang-1300.0.29.30)]"
OS,posix [darwin]
Cython,3.0.11


<h1> Prepare the simulation </h1>

In [4]:
def generate_data_2d(L, lenght_mirror, n_A, sigma, x_0, y_0, g_0, omega_A, omega_0, omega_max, T, dt, position_basis=None):
    ###Problem parameters

    #position of the atoms
    print("Preparing the position of the atoms ...")
    spacing_atom = lenght_mirror / (np.sqrt(2)**2 * n_A)
    position_atom_list = np.zeros((n_A, 2))
    position_atom_list[n_A//2] = [0,0]
    
    for i in range(1, n_A//2+1):
        position_atom_list[n_A//2 - i] = [-i*spacing_atom, -i*spacing_atom]
        position_atom_list[n_A//2 + i] = [i*spacing_atom, i*spacing_atom]

    #prepare the momentum space
    dp = 2*pi/L
    omega_tab_p = np.array([dp*n for n in range(1000000) if dp*n <= omega_max])
    kx_tab = np.sort(np.unique(np.concatenate((-omega_tab_p, omega_tab_p))))
    ky_tab = np.copy(kx_tab)
    n_modes = len(kx_tab)
    print("Number of modes in momentum space: ", n_modes, " X ", n_modes)
    dim_subspace = n_modes**2 + n_A

    ### Encode the Hamiltonian
    H_0_matrix = np.zeros((dim_subspace, dim_subspace), dtype=complex)
    H_I_matrix = np.zeros((dim_subspace, dim_subspace), dtype=complex)

    #diagonal elements
    dummy_index = 0
    for i in range(n_modes):
        for j in range(n_modes):
            H_0_matrix[dummy_index, dummy_index] = np.sqrt(kx_tab[i]**2 + ky_tab[j]**2)
            dummy_index += 1

    for j in range(n_A):
        H_I_matrix[n_modes**2+j, n_modes**2+j] = omega_A

    #off-diagonal elements
    dummy_index_0 = 0
    for i in range(n_modes):
        for j in range(n_modes):
            dummy_index_1 = n_modes**2
            for k in range(n_A):
                position_atom = position_atom_list[k]
                k_vector = np.array([kx_tab[i], ky_tab[j]])
                H_I_matrix[dummy_index_1, dummy_index_0] = 1j*g_0*np.sqrt(omega_A)/L * np.exp(-1j*np.dot(k_vector, position_atom))
                H_I_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_I_matrix[dummy_index_1, dummy_index_0])
                dummy_index_1 += 1

            dummy_index_0 += 1

    H_0 = qt.Qobj(H_0_matrix)
    H_I = qt.Qobj(H_I_matrix)

    ###Encode the initial state in qutip
    print("Preparing the initial state ...")
    coef_momentum = np.zeros((n_modes, n_modes), dtype=complex)
    k_bar = np.array([omega_0, 0])
    r_bar = np.array([x_0, y_0])

    init_state = 0
    dummy_index = 0
    for i in range(n_modes):
        for j in range(n_modes):
            k_vector = np.array([kx_tab[i], ky_tab[j]])
            coef_momentum[i, j] = np.exp(-0.5*sigma**2*np.linalg.norm(k_vector - k_bar)**2 - 1j*np.dot(k_vector, r_bar))
            init_state += coef_momentum[i, j]*qt.basis(dim_subspace, dummy_index)
            dummy_index += 1
    init_state = init_state.unit()


    ###Now conduct the time evolution
    print("Starting the time evolution ...")
    n_step = int(T/dt)
    t_list = np.linspace(0, T, n_step)
    state_list = []
    state_list.append(init_state)

    for t in tqdm(range(1, n_step)):
        current_state = state_list[-1]
        new_state = (-1j * H_0 *dt - H_0**2 * dt**2 ) @ current_state
        new_state = (-1j*H_I*dt - H_I**2 * dt**2) @ new_state
        state_list.append(qt.Qobj(new_state).unit())

    ###Compute the quantities of interest
    N_k_t = np.zeros((n_modes, n_modes, n_step))
    N_A_t = np.zeros((n_step, 1))
    
    print("Computing occupation numbers in momentum space...")
    for t in tqdm(range(n_step)):
        current_state = state_list[t]

        dummy_index = 0
        for i in range(n_modes):
            for j in range(n_modes):
                momentum_state = qt.basis(dim_subspace, dummy_index)
                N_k_t[i, j, t] = np.abs(momentum_state.dag() * current_state)**2
                dummy_index += 1

        for k in range(n_A):
            atomic_state = qt.basis(dim_subspace, n_modes**2+k)
            N_A_t[t] += np.abs(atomic_state.dag() * current_state)**2
    
    if position_basis is None:
        N_x_t = None
    else:
        N = position_basis.shape[0]
        N_x_t = np.zeros((N, N, n_step))
        print("Computing occupation numbers in position space...")
        for t in tqdm(range(n_step)):
            current_state = state_list[t]
            for i in range(N):
                for j in range(N):
                    position_state = qt.Qobj(position_basis[i, j])
                    N_x_t[i, j, t] = np.abs(position_state.dag() * current_state)**2
    
    ###Compute reflection and transmission coefficients
    P_x_t = np.zeros((n_step, 1))
    P_y_t = np.zeros((n_step, 1))
    for t in range(n_step):
        for i in range(n_modes):
            for j in range(n_modes):
                if kx_tab[i] > ky_tab[j]:
                    P_x_t[t] += N_k_t[i, j, t]
                else:
                    P_y_t[t] += N_k_t[i, j, t]
    
    return t_list, position_atom_list, N_k_t, N_x_t, N_A_t, P_x_t, P_y_t

In [5]:
L = 10*pi
T = L/2
dt = 0.1

#width of the wave packet
sigma = L/16
sigma_momentum = 1/sigma
x_0 = -L/4


#Jaynes-Cummings parameter g_0 and energies 
g_0 = 0.1
omega_A = 5
gamma = 2*pi*g_0**2*omega_A**2

omega_0_m = omega_A - gamma
omega_0_p = omega_A + gamma

#resolution of the mirror 
n_A = 19
lenght_mirror = np.sqrt(2)*L / 2
spacing_prediction = lenght_mirror / n_A
wave_lenght_photon = 2*pi / omega_A
print("Spacing between atoms: ", spacing_prediction)
print("Wave length of the photons at resonance: ", wave_lenght_photon)
print("----------------------------------")

print("Delta_k / Gamma: ", sigma_momentum/gamma)

value_to_check = g_0*np.sqrt(omega_A)/L
if value_to_check > 0.1:
    print("Warning: coupling too large")
print("Coupling g: ", g_0*np.sqrt(omega_A)/L)
print("omega_0_m = ", omega_0_m, " and omega_0_p = ", omega_0_p)

Spacing between atoms:  1.169179720567991
Wave length of the photons at resonance:  1.2566370614359172
----------------------------------
Delta_k / Gamma:  0.3242277876554808
Coupling g:  0.007117625434171771
omega_0_m =  3.4292036732051034  and omega_0_p =  6.570796326794897


In [6]:
#define momentum space parameters
omega_0 = 5
omega_max = 2.5*omega_A
x_0 = -L/4
y_0 = 0

T = L/2
dt = 0.1


#ONLY IF NEEDED : position basis
N = 64
generate_position_basis = False

if generate_position_basis:
    
    #get back the momentum space basis
    dp = 2*pi/L
    omega_tab_p = np.array([dp*n for n in range(1000000) if dp*n <= omega_max])
    kx_tab = np.sort(np.unique(np.concatenate((-omega_tab_p, omega_tab_p))))
    ky_tab = np.copy(kx_tab)
    n_modes = len(kx_tab)
    dim_subspace = n_modes**2 + n_A

    #prepare the position basis
    print("Preparing the position basis ...")
    x = np.linspace(-L/2, L/2, N)
    y = np.linspace(-L/2, L/2, N)
    position_basis = np.zeros((N, N, dim_subspace ), dtype=complex)
    pbar = tqdm(total=N**2)
    for i in range(N):
        for j in range(N):
            position_state = 0
            dummy_index = 0
            for a in range(n_modes):
                for b in range(n_modes):
                    k_vector = np.array([kx_tab[a], ky_tab[b]])
                    position_state += np.exp(-1j*np.dot(k_vector, [x[i], y[j]])) * qt.basis(dim_subspace, dummy_index)
                    dummy_index += 1
            position_state = position_state.unit()
            position_basis[i, j] = position_state.full().flatten()
            pbar.update(1)

    pbar.close()
    

In [ ]:
if generate_position_basis:
    t_list, position_atom_list, N_k_t, N_x_t, N_A_t, P_x_t, P_y_t = generate_data_2d(L, lenght_mirror, n_A, sigma, x_0, y_0, g_0, omega_A, omega_0, omega_max, T, dt, position_basis=position_basis)
else:
    t_list, position_atom_list, N_k_t, N_x_t, N_A_t, P_x_t, P_y_t = generate_data_2d(L, lenght_mirror, n_A, sigma, x_0, y_0, g_0, omega_A, omega_0, omega_max, T, dt)

Preparing the position of the atoms ...
Number of modes in momentum space:  125  X  125
Preparing the initial state ...
Starting the time evolution ...


  0%|          | 0/156 [00:00<?, ?it/s]